# 準備

In [1]:
import getpass
import os

# OpenAI API キーの設定
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

OpenAI API キーを入力してください:  ········


# テキスト生成の基本的な流れ

In [3]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    temperature=0.0,
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "こんにちは"}],
)

print(response.choices[0].message.content)

こんにちは！今日はどんなことをお話ししましょうか？


# シンプルな対話 AI の作成

In [12]:
history = []
n = 10  # 会話の上限
model = "gpt-4o-mini"
for _ in range(n):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    print(f"ユーザ: {user_input}")
    history.append({"role": "user", "content": user_input})
    response = client.chat.completions.create(model=model, messages=history)
    content = response.choices[0].message.content
    print(f"AI: {content}")
    history.append({"role": "assistant", "content": content})

ユーザ入力:  こんにちは


ユーザ: こんにちは
AI: こんにちは！今日はどんなことをお話ししましょうか？


ユーザ入力:  exit


# テキスト生成の応用

# 生成できた部分から順に表示する

In [ ]:
history = []
n = 10  # 会話の上限
model = "gpt-4o-mini"
for _ in range(n):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    print(f"ユーザ: {user_input}")
    history.append({"role": "user", "content": user_input})
    # stream=True でストリーミングを有効化
    stream = client.chat.completions.create(model=model, messages=history, stream=True)
    print("AI: ", end="")
    # 応答を集める文字列
    ai_content = ""
    # ストリーミングの各チャンクを処理
    for chunk in stream:
        # message ではなく ChoiceDelta
        content = chunk.choices[0].delta.content
        # ChoiceDelta の finish_reason が stop なら生成完了
        if chunk.choices[0].finish_reason == "stop":
            break
        print(content, end="")
        ai_content += content
    print()
    history.append({"role": "assistant", "content": ai_content})

# 最大公約数を求めるツールの利用

In [18]:
gcd_function = {
    "name": "gcd",
    "description": "最大公約数を求める",
    "parameters": {
        "type": "object",
        "properties": {
            "num1": {"type": "number", "description": "整数1"},
            "num2": {"type": "number", "description": "整数2"},
        },
        "required": ["num1", "num2"],
    },
}
tools = [{"type": "function", "function": gcd_function}]

messages = [
    {"role": "user", "content": "50141 と 53599 の最大公約数を求めてください。"}
]

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages, tools=tools
)
print(response.choices[0].message.content)  # None
print(response.choices[0].finish_reason)  # tool_calls
print(response.choices[0].message.tool_calls)  # [ChatCompletionMessageToolCall(...)]


None
tool_calls
[ChatCompletionMessageToolCall(id='call_DduSimZWU8V8CWhklx0P1f0W', function=Function(arguments='{"num1":50141,"num2":53599}', name='gcd'), type='function')]


# 関数情報を抽出

In [20]:
import json

function_info = response.choices[0].message.tool_calls[0].function
name = function_info.name
args = json.loads(function_info.arguments)

# 最大公約数の計算

In [24]:
import math

print(math.gcd(args["num1"], args["num2"]))

1729


# Pydantic を用いた関数の定義

In [34]:
from pydantic import BaseModel, Field
import json
import math

class GCD(BaseModel):
    num1: int = Field(description="整数1")
    num2: int = Field(description="整数2")


gcd_function = {
    "name": "gcd",
    "description": "最大公約数を求める",
    "parameters": GCD.model_json_schema(),
}


function_info = response.choices[0].message.tool_calls[0].function
name = function_info.name
args = json.loads(function_info.arguments)

print(math.gcd(args["num1"], args["num2"]))

1729


# Pydantic を用いた引数の取得

In [38]:
parsed_result = GCD.model_validate_json(
    response.choices[0].message.tool_calls[0].function.arguments
)
print(parsed_result)

num1=50141 num2=53599


In [46]:
class LCM(BaseModel):
    num1: int = Field(description="整数1")
    num2: int = Field(description="整数2")


lcm_function = {
    "name": "lcm",
    "description": "最小公倍数を求める",
    "parameters": LCM.model_json_schema(),
}

tools = [
    {"type": "function", "function": gcd_function},
    {"type": "function", "function": lcm_function},
]

messages = [
    {
        "role": "user",
        "content": "50141 と 53599 の最大公約数と最小公倍数を求めてください。",
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages, tools=tools
)
choice = response.choices[0]
if choice.finish_reason == "tool_calls":
    for tool in choice.message.tool_calls:
        if tool.function.name == "gcd":
            gcd_args = GCD.model_validate_json(tool.function.arguments)
            print(f"最大公約数: {math.gcd(gcd_args.num1, gcd_args.num2)}")
        elif tool.function.name == "lcm":
            lcm_args = LCM.model_validate_json(tool.function.arguments)
            print(f"最小公倍数: {math.lcm(lcm_args.num1, lcm_args.num2)}")
elif choice.finish_reason == "stop":
    print("AI: ", choice.message.content)

最大公約数: 1729
最小公倍数: 1554371


# response_format の利用例

In [49]:
class Translations(BaseModel):
    english: str = Field(description="英語の文章")
    french: str = Field(description="フランス語の文章")
    chinese: str = Field(description="中国語の文章")


prompt = f"""\
以下に示す文章を英語・フランス語・中国語に翻訳してください。
ただし、アウトプットは後述するフォーマットの JSON 形式で出力してください。

# 文章
吾輩は猫である。名前はまだない。

# 出力フォーマット
以下に JSON Schema 形式のフォーマットを示します。このフォーマットに従うオブジェクトの形で出力してください。
{Translations.model_json_schema()}
"""

response = client.chat.completions.create(
    temperature=0.0,
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    response_format={"type": "json_object"},
)

translations = Translations.model_validate_json(response.choices[0].message.content)
print("英語:", translations.english)
print("フランス語:", translations.french)
print("中国語:", translations.chinese)

英語: I am a cat. I do not have a name yet.
フランス語: Je suis un chat. Je n'ai pas encore de nom.
中国語: 我是一只猫。还没有名字。


# 画像を入力する

In [63]:
import base64
from pathlib import Path
from typing import Any

from openai import OpenAI

client = OpenAI()


def image2content(image_path: Path) -> dict[str, Any]:
    # base64 エンコード
    with image_path.open("rb") as f:
        image_base64 = base64.b64encode(f.read()).decode("utf-8")

    # content の作成
    content = {
        "type": "image_url",
        "image_url": {"url": f"data:image/png;base64,{image_base64}", "detail": "low"},
    }
    return content

prompt = "これは何の画像ですか?"
image_path = Path("./sample_image1.png")
contents = [{"type": "text", "text": prompt}, image2content(image_path)]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.0,
    messages=[{"role": "user", "content": contents}],
)

print(response.choices[0].message.content)

これは猫のイラストです。シンプルな線画で描かれており、猫の特徴が表現されています。


In [69]:
image_path2 = Path("./sample_image2.png")

prompt = "2枚の画像の違いを教えてください。"
contents = [
    {"type": "text", "text": prompt},
    image2content(image_path),
    image2content(image_path2),
]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.0,
    messages=[{"role": "user", "content": contents}],
)

print(response.choices[0].message.content)

1枚目の画像は、シンプルな線画スタイルの猫のイラストで、表情や形が抽象的です。2枚目の画像は、カラフルで詳細なイラストの猫で、帽子をかぶり、背景に円形のデザインがあります。

主な違いは、スタイル（線画 vs. カラフルなイラスト）、ディテールの豊富さ、そして背景の有無です。


# 音声を文字起こしする。

In [73]:
from pathlib import Path
from openai import OpenAI


client = OpenAI()
audio_path = Path("./sample_audio.mp3")

with audio_path.open("rb") as f:
    transcription = client.audio.transcriptions.create(
        model="whisper-1", file=f, temperature=0.0
    )
print(transcription.text)

私の名前は下ヶ内です。


# プロンプトを用いて文字起こしする。

In [80]:
prompt = "下垣内"

with audio_path.open("rb") as f:
    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=f,
        prompt=prompt,
        response_format="text",
        temperature=0.0,
    )
print(transcription)

私の名前は下垣内です。



# 音声を合成する。

In [92]:
audio_output_path = Path("output.mp3")
with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="alloy",
    input="こんにちは。私は AI アシスタントです！",
) as response:
    response.stream_to_file(audio_output_path)

# 画像を生成する

In [95]:
from openai import OpenAI
import requests

client = OpenAI()

prompt = """\
メタリックな球体
"""

response = client.images.generate(
    model="dall-e-3", prompt=prompt, n=1, size="1024x1024"
)

image_url = response.data[0].url
image = requests.get(image_url).content
with open("output1.png", "wb") as f:
    f.write(image)

In [97]:
response = client.images.generate(
    model="dall-e-3", prompt=prompt, n=1, size="1024x1024", response_format="b64_json"
)

image = response.data[0].b64_json

with open("output2.png", "wb") as f:
    f.write(base64.b64decode(image))

In [99]:
print(response.data[0].revised_prompt)

A sphere with a metallic texture reflecting the environment around it
